## Untersuchen Tournament data

In [2]:
import os
import pandas as pd

In [3]:
#Verzeichnis anpassen

os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
print(os.getcwd())

C:\Users\Katharina\Desktop\Weiterbildung\Bootcamp\Bootcamp\Final_project\data


In [ ]:
# tourn1 = pd.read_csv('df_clean.csv', sep=';')
# tourn2 = pd.read_csv('df_clean2.csv', sep=';')
# tourn3 = pd.read_csv('clean45.csv', sep=';')
# tourn4 = pd.read_csv('clean5.csv', sep=';')
# tourn5 = pd.read_csv('Tournaments7575_7576.csv', sep=';')


tourn1

In [15]:
Df_tourn1 = pd.read_csv('Tournaments2024_all.csv', sep=';')
Df_tourn2 = pd.read_csv('Tournaments2024_all_part2.csv', sep=';')

# DF_tournament_all = pd.concat([Df_tourn1, Df_tourn2])
# DF_tournament_all['No'] = DF_tournament_all['No'].astype(str)
# cols = ['Name','DefaultCity', 'No','Title', 'StartDate', 'StartDateMainDraw',  'EndDateMainDraw', 'CountryName']
# DF_tournament_short = DF_tournament_all.loc[:,cols]

tourn2

In [16]:
tourn2 = pd.read_csv('df_clean2.csv', sep=';')

tourn3 

In [17]:
testT1 = pd.read_csv('test_tournament6000_6400.csv', sep=';')
testT2 = pd.read_csv('test_tournament6400_6900.csv', sep=';')
testT3 = pd.read_csv('test_tournament6900_7481.csv', sep=';')

tourn4  == clean5

In [18]:
testT4 = pd.read_csv('test_tournament4000_5000.csv', sep=';')
testT5 = pd.read_csv('test_tournament5000_6000.csv', sep=';')
testT6 = pd.read_csv('test_tournament6000.csv', sep=';')


In [19]:
## alle zusammenfügen
Dat_comp = pd.concat([Df_tourn1, Df_tourn2, tourn2, testT1, testT2, testT3, testT4, testT5, testT6])

In [20]:
Dat_comp2 = Dat_comp.drop_duplicates(subset=['No'])

In [23]:
#anhand von type Tournaments auswählen

filter1 = [4,5,6,10,32,33,38,39,40,41,42,51,52,53]
#clean44 = clean4.loc[clean4['Type'].isin(filter1)]
Dat_comp3 = Dat_comp2.loc[Dat_comp2['Type'].isin(filter1)]

In [24]:
Dat_comp3.to_csv('Alle_Tuniere.csv', index=False, sep=';')

In [26]:
Dat_comp4 = Dat_comp3.copy()
Dat_comp4['No'] = Dat_comp4['No'].astype(str)

## vergleichen mit den statistiken 

In [28]:
import glob
# Alle passenden CSV-Dateien im aktuellen Verzeichnis finden
csv_files = glob.glob("tournament_statistics_*.csv")

In [32]:
import re
# Extrahiere die Nummern aus den Dateinamen
nummern = [re.search(r'_(\d+)\.csv', name).group(1) for name in csv_files]

# Prüfen, ob diese Nummern in der DataFrame-Spalte vorkommen
Dat_comp4['check'] = Dat_comp4['No'].isin(nummern)

In [ ]:
#Gecancellte TUniere entfernen
Dat_comp5 = Dat_comp4.copy()
Dat_comp5 = Dat_comp5[~Dat_comp5['Title'].str.contains('cancelled', case=False, na=False)]
Dat_comp6 = Dat_comp5[~Dat_comp5['Title'].str.contains('Canceled', case=False, na=False)]


In [43]:
Dat_comp7 = Dat_comp6.copy()
Dat_comp7['check'] = Dat_comp7['check'].astype(bool)
Dat_comp7 = Dat_comp7[~((Dat_comp7['StartDate'].str.startswith('2025')) & (Dat_comp7['check'] == False))]

In [46]:
fehlendeTuniere = Dat_comp7[Dat_comp7['check'] == False]
fehlendeTuniere_NO = fehlendeTuniere['No']

In [48]:
fehlendeTuniere_NO.to_csv('missingTournaments.csv', index=False, sep=';')

##### vergleichenmit den alten csv-files

In [49]:
csv2 = glob.glob("tournament_statistics_*.csv")

In [ ]:
#vergleich 
fehlend_in_liste2 = list(set(csv2) - set(csv_files))
fehlend_in_liste2

#56 neue tuniere

In [58]:
nummern2 = [re.search(r'_(\d+)\.csv', name).group(1) for name in csv2]

# Prüfen, ob diese Nummern in der DataFrame-Spalte vorkommen
Dat_comp7['check2'] = Dat_comp7['No'].isin(nummern2)


In [59]:
#Tuniere finalisieren
Dat_comp8 = Dat_comp7.copy()
Dat_comp8 = Dat_comp8[~Dat_comp8['check2']==False]

In [60]:
Dat_comp8.to_csv('Alle_Tuniere_final.csv', index=False, sep=';')

### Daten zusammenführen

In [16]:
Tuniere = pd.read_csv('Alle_Tuniere_final.csv', sep=';')

In [17]:
Tuniere1 = Tuniere.copy()
Tuniere1['No'] = Tuniere1['No'].astype(str)

In [ ]:
import glob
import re
# CSV-Dateien im aktuellen Verzeichnis finden
csv_files = glob.glob("tournament_statistics_*.csv")

# Zwei leere Listen für die Gruppen
df_list_30 = []
df_list_mixed = []

for file in csv_files:
    df = pd.read_csv(file, sep='\t')
    
    # Turniernummer aus Dateinamen extrahieren
    match = re.search(r'tournament_statistics_(\d+)\.csv', os.path.basename(file))
    if match:
        df["TournamentNo"] = match.group(1)
    else:
        df["TournamentNo"] = None

    # Unterscheidung nach Typ
    if 'ItemType' in df.columns:
        types = df['ItemType'].astype(str).unique()
        if set(types) == {'30'}:
            df_list_30.append(df)
        elif '11' in types:
            df_list_mixed.append(df)
        else:
            print(f"Unerwartete Typen in {file}: {types}")
    else:
        print(f"'Type' column missing in {file}, skipping...")

# Zu einem DataFrame zusammenfügen
combined_30 = pd.concat(df_list_30, ignore_index=True)
combined_mixed = pd.concat(df_list_mixed, ignore_index=True)

Daten_all = pd.concat([combined_30, combined_mixed], ignore_index=True, sort=False)
#object

In [14]:
Daten_all1 = Daten_all.copy()
Daten_all1['NoMatch'] = Daten_all1['NoMatch'].astype(str)

In [11]:
BeachMatches = pd.read_csv('BeachMatchList_full.csv', sep=';') #int

In [12]:
BeachMatches1 = BeachMatches.copy()
BeachMatches1['@No'] = BeachMatches1['@No'].astype(str)

In [15]:
#Zusammenführen von Spielstatistiken und Matches

Combined_Stat = Daten_all1.merge(BeachMatches1, left_on = 'NoMatch', right_on='@No', how= 'left')

In [ ]:
#mit Tournaments mergen

#Dat_Merge = Combined_Stat.merge(Tuniere1, left_on='TournamentNo', right_on='No', how='left')

In [21]:
Dat_Merge1 = Tuniere1.merge(Combined_Stat, right_on='TournamentNo', left_on='No', how='left')

In [ ]:
#DatNo = Dat_Merge1[['NoMatch', '@TeamAName', '@TeamBName']]

In [ ]:
#DatNo.to_csv('FehlInfo.csv', index=False, sep=';')

### Daten auf Koordinaten vorbereiten

In [22]:
#1. Aufteilen in City befüllt und City nicht befüllt
Dat_Merge1['DefaultCity'].value_counts()

CityNA = Dat_Merge1[Dat_Merge1['DefaultCity'].isna()]
City_Full = Dat_Merge1[~Dat_Merge1['DefaultCity'].isna()]

In [29]:
#in Name_x ist Ort enthalten --> in DefaultCity schreiben
#CityNA[['Title', 'Name_x']].drop_duplicates()
CityNA1 = CityNA.copy()
CityNA1['DefaultCity'] = CityNA1['Name_x']

In [33]:
import pandas as pd
import numpy as np

# Angenommen, dein DataFrame heißt df und die Spalte mit Turniernamen heißt "TournamentName"
df = CityNA1.copy()#.dropna(subset=["Name_y"])  # NaN entfernen, falls vorhanden


# Schritt 1: Alle Einträge mit **nur einem Wort** (keine Leerzeichen)
df["Ort"] = pd.Series(dtype="object")  # Neue Spalte initialisieren

maske_ein_wort = df["DefaultCity"].str.strip().str.count(r'\s') == 0
df.loc[maske_ein_wort, "Ort"] = df.loc[maske_ein_wort, "DefaultCity"].str.strip()

# Schritt 2: Aus den übrigen Zeilen – Ort nach Bindestrich extrahieren
# Nur Zeilen ohne bereits gesetzten Ort
maske_mit_bindestrich = df["Ort"].isna() & df["DefaultCity"].str.contains(" - ")

# Regex: alles nach dem letzten " - " wird als Ort interpretiert
df.loc[maske_mit_bindestrich, "Ort"] = df.loc[maske_mit_bindestrich, "DefaultCity"].str.extract(r" - ([^-\n\r]+)$")[0].str.strip()

# Optional: Die Zeilen, die keinen Ort haben, explizit mit 'Unbekannt' befüllen (oder eine andere Default-Wert)
df["Ort"] = df["Ort"].fillna("Unbekannt")

# Optional: Duplikate in der Ort-Spalte beibehalten (keine Entfernung)
df_mit_ort = df[df["Ort"].notna()].copy()


In [34]:
df_mit_ort[['Ort', 'DefaultCity']].drop_duplicates()

,Ort,DefaultCity
19597,Ljubljana,Ljubljana
20717,Balıkesir,Balıkesir
20941,Bialystok,Bialystok
21165,Unbekannt,Giardini Naxos
21389,Lecce,Lecce
...,...,...
88462,1st event,Cancun Hub - 1st event
88910,2nd event,Cancun Hub - 2nd event
89358,3rd event,Cancun Hub - 3rd event
89806,Sochi,Sochi


In [35]:
#ID anfügen

#Id anfügen
df_mit_ort['ID_row'] = range(1, len(df_mit_ort) +1)
df_mit_ort['ID_DefaultCity'] = df_mit_ort['DefaultCity'].astype('category').cat.codes

In [38]:
ort_adapt = df_mit_ort[['Ort', 'DefaultCity', 'ID_DefaultCity']].drop_duplicates()

In [39]:
ort_adapt.to_csv('Ort_Noch_anpassen.csv', index=False, sep=';')

In [42]:
#bearbeitete Datei wieder einlesen
Dat_ort = pd.read_csv('Ort_Noch_anpassen.csv', sep=';')

In [44]:
#Datei wieder anfügen
dat_ortF = df_mit_ort.merge(Dat_ort.iloc[:, [0, 2]], left_on='ID_DefaultCity', right_on='ID_DefaultCity', how='left')

#Ort Neu angefügt
dat_ortF2 = dat_ortF.copy()


In [45]:
dat_ortF2['DefaultCity'] = dat_ortF2['Ort_Neu']

In [48]:
#neue variablen entfernen
dat_ortF3 = dat_ortF2.copy()
dat_ortF3 = dat_ortF3.drop(['Ort', 'ID_row', 'ID_DefaultCity', 'Ort_Neu'], axis=1)

In [49]:
#Daten wieder zusammenfügen
DatFinal = pd.concat([City_Full, dat_ortF3],ignore_index=True)


In [50]:
DatFinal.to_csv('Datenbestand_Final.csv', index=False, sep=';')

In [ ]:
#es gibt ein missing bei localtime -> entfernen

In [51]:
DatFinal1 = DatFinal[DatFinal['@LocalTime'].notna()]

In [52]:
DatFinal.to_csv('Datenbestand_Final1.csv', index=False, sep=';')